In [1]:
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
from datasets import Dataset
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-23 22:35:46 __init__.py:186] Automatically detected platform cuda.


In [2]:
model_name = "ExplosionNuclear/Llama-3.2-3B-bnb-4bit-checkpoints"
revision_id = "ca175a01817db5132d07052ce0b6aee0f341f061" # 720

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
model_name = "ExplosionNuclear/Llama-3.2-3B-bnb-4bit-checkpoints",
revision=revision_id,
max_seq_length = 800,
dtype = None,
load_in_4bit = True)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.dev0.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.676 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.6. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
model.config.output_hidden_states = True

In [5]:
from datasets import Dataset
test_dataset = Dataset.load_from_disk("/workspace/experiments/MATS/data/test/test_dataset_0_100")
print(test_dataset)

Dataset({
    features: ['question', 'answer', 'simple_talk', 'full_answer', 'simple_question', 'pattern'],
    num_rows: 89
})


In [6]:
model = FastLanguageModel.for_inference(model)

In [ ]:
test_dataset[30]['question']

In [6]:
n = 39
inputs = tokenizer(
    [
        test_dataset[n]['simple_question'] + '\n' + test_dataset[n]['simple_talk']
    ], return_tensors = "pt").to("cuda")

inputs['input_ids'][0]

tensor([128000,  37310,    922,    279,  44578,   2477,     13,   2360,    810,
          1243,    220,     22,  23719,    627,   1451,    648,   2477,    374,
           264,  17779,    315,   4731,  32971,    555,   1202,    781,  37731,
          5788,     11,  26609,   9262,  24142,     11,    323,   3629,     11,
           264,  32558,  85339,     13,   1102,  22763,    304,    279,    220,
          3753,     15,     82,    304,    279,   6560,    323,   2326,     11,
         28160,    555,   1772,   2320,   3200,    323,    502,  12330,     13,
           578,  17779,  18661,  23354,    304,    279,    220,   2550,     15,
            82,    449,  21562,   1093,  43856,   1133,     11,  12433,    291,
           555,  83426,     11,    323,   2893,    360,   1494,     13,  77439,
          2477,    374,   3629,   5938,    449,    264,   2512,  10816,     11,
         83374,  16975,    323,    264,   5357,    389,  62684,    323,  16975,
           927,   6485,   5609,  14726, 

In [69]:
print(test_dataset[n]['pattern'])

[1451, 648, 2477, 374, 264, 17779, 315, 4731, 32971, 555, 1202, 781, 37731, 5788, 11, 26609, 9262, 24142, 11, 323, 3629, 11, 264, 32558, 85339, 13, 1102, 22763, 304, 279, 220, 3753, 15, 82, 304, 279, 6560, 323, 2326, 11, 28160, 555, 1772, 2320, 3200, 323, 502, 12330, 13, 578, 17779, 18661, 23354, 304, 279, 220, 2550, 15, 82, 449, 21562, 1093, 43856, 1133, 11, 12433, 291, 555, 83426, 11, 323, 2893, 360, 1494, 13, 77439, 2477, 374, 3629, 5938, 449, 264, 2512, 10816, 11, 83374, 16975, 323, 264, 5357, 389, 62684, 323, 16975, 927, 6485, 5609, 14726, 13, 578, 17779, 706, 8738, 311, 38680, 927, 279, 1667, 11, 52913, 5540, 315, 14683, 323, 29036, 4731, 13, 4427, 28289, 44578, 2477, 21562, 2997, 65307, 6785, 11, 578, 55317, 5475, 11, 323, 800, 13, 35407, 13, 77439, 2477, 706, 1101, 1027, 37475, 369, 1202, 5845, 311, 1893, 264, 5647, 315, 66264, 323, 34104, 11, 3629, 1555, 279, 1005, 315, 45166, 24198, 323, 4382, 11, 3686, 7524, 11, 28904, 13, 28993, 11, 44578, 2477, 374, 264, 17779, 430, 2819, 

In [7]:
import re

def extract_number(text):
    match = re.search(r'(\d+)!<\|end_of_text\|>$', text)
    return int(match.group(1)) if match else None

def extract_right_answer(text):
    match = re.search(r'(\d+)!$', text)
    return int(match.group(1)) if match else None

def begin_sq(inputs, pattern):

    pattern = torch.tensor(pattern).to("cuda")
    windows = inputs.unfold(0, pattern.numel(), 1)
    matches = (windows == pattern).all(dim=1)
    indices = torch.nonzero(matches).flatten().tolist()
    begin = indices[0]

    return begin


In [70]:
n = 39
inputs = tokenizer(
    [
        test_dataset[n]['question']
    ], return_tensors = "pt").to("cuda")

find_simple_talk_begin(inputs['input_ids'][0], test_dataset[n]['pattern'][0:10]) 


292

In [13]:
probabilities = []
import numpy as np

def accuracy(data, n):
    
    results = {}

    # cycle through a given set of test tasks with
    for idx, example in enumerate(data):

        inputs = tokenizer(
            [
                example['question']
            ], return_tensors = "pt").to("cuda")

        solutions = []
        right_answer = extract_right_answer(example['answer'])
        

        # generate n answers to see the distribution of right answers
        n_runs = 0
        for k in range(n):
            
            outputs = model.generate(**inputs, max_new_tokens = 800, use_cache = True)
            text = tokenizer.batch_decode(outputs)[0].split("And.. ")[-1]
            number = extract_number(text)

            if number != None:
                solutions.append(number)

                """
                if number == right_answer:

                    begin = begin_sq(inputs['input_ids'][0], example['pattern'][0:10])

                    get residual streams and compare with 
                    residual streams for simple talks
                    
                    outputs = model(**inputs)
                    residual1 = outputs.hidden_states
                    
                    inputs_simple = tokenizer(
                   [
                        example['simple_question'] + '\n' + example['simple_talk']
                    ], return_tensors = "pt").to("cuda")
                    
                    outputs_simple = model(**inputs_simple)
                    residual2 = outputs_simple.hidden_states

                    #loop for layers (29)
                    layer_norms = []
                    for res1, res2 in zip(residual1, residual2):
                        # res [1, N, 30902]
                        diff = res1 - res2 # [1, N, 30902]
                        l = len(example['pattern'])
                        norms = diff[:, begin:begin + l, :].norm(dim=2) #[1, l]
                        norms = norms.squeeze(0) # [l]
                        layer_norms.append(norms)

                    
                    current_run = torch.stack(layer_norms, dim=0)
                    if accumulated_norms is None:
                        accumulated_norms = current_run
                    else:
                        accumulated_norms += current_run
                        
                    n_runs += 1

        try:
            mean_norms = accumulated_norms / n_runs
            max_values = [t.max().item() for t in mean_norms]
            min_values = [t.max().item() for t in mean_norms]
            max_min = [(a,b) for a, b in zip(max_values, min_values)]
            res[f'diffs_for_{idx}'] = max_min
        except:
            continue
        """
                
        probabilities.append(solutions.count(right_answer) / n)

    return probabilities #resdiff

In [ ]:
percentage = [0.9, 0.7, 0.5, 0.4, 0.3, 0.2, 0.1, 0.01]

maxs = []
means = []

for idx, percent in enumerate(percentage):
    print('dataset', idx)
    test_dataset = Dataset.load_from_disk(f"/workspace/experiments/MATS/data/test/test_dataset_{idx}_100")
    probabilities = accuracy(test_dataset, 100)
    pmax = np.array(probabilities).max()
    pmean = np.array(probabilities).mean()
    maxs.append(pmax)
    means.append(pmean)
    print('max', pmax)
    print('mean', pmean)


dataset 0


In [15]:
probabilities

[1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0]